In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from fpdf import FPDF
from datetime import datetime

fear_file = "/content/fear_greed_index.csv"
hist_file = "/content/historical_data.csv"
out_dir = "./outputs"
csv_out_dir = "./csv_files"
pdf_path = os.path.join(out_dir, "ds_report.pdf")

os.makedirs(out_dir, exist_ok=True)
os.makedirs(csv_out_dir, exist_ok=True)

def norm_cols(df):
    df.columns = [str(c).strip().lower().replace(" ", "_").replace("-", "_").replace(".", "_") for c in df.columns]
    return df

def find_col_by_candidates(cols, candidates):
    cols = [c.lower() for c in cols]
    for cand in candidates:
        if cand in cols:
            return [c for c in cols if c==cand][0]
    for cand in candidates:
        for c in cols:
            if cand in c:
                return c
    return None

fg = pd.read_csv(fear_file)
hist = pd.read_csv(hist_file)
print("Loaded FG:", fg.shape, "HIST:", hist.shape)

fg = norm_cols(fg)
hist = norm_cols(hist)

fg_date_col = find_col_by_candidates(fg.columns, ["date","timestamp","time","day"])
hist_date_col = find_col_by_candidates(hist.columns, ["timestamp","time","date","trade_time","ts"])

if fg_date_col is None or hist_date_col is None:
    raise ValueError("Could not find date/time columns.")

fg[fg_date_col] = pd.to_datetime(fg[fg_date_col], errors="coerce", infer_datetime_format=True)
hist[hist_date_col] = pd.to_datetime(hist[hist_date_col], errors="coerce", infer_datetime_format=True)

sent_col = find_col_by_candidates(fg.columns, ["classification", "sentiment", "label", "fear_greed"])
if sent_col is None:
    for c in fg.columns:
        vals = fg[c].dropna().astype(str).str.lower().unique()[:10]
        if any(v in ["fear", "greed"] for v in vals):
            sent_col = c
            break
if sent_col is None:
    raise ValueError("Could not detect sentiment/classification column.")

fg = fg.rename(columns={fg_date_col: "date", sent_col: "classification"})
fg["classification"] = fg["classification"].astype(str).str.strip().str.capitalize()
fg = fg[fg["classification"].isin(["Fear", "Greed"])].copy()
print("FG sentiment counts:\n", fg["classification"].value_counts())

mappings = {
    "account": ["account","acct","user","trader"],
    "symbol": ["symbol","pair","ticker","coin"],
    "execution_price": ["execution_price","price","executionprice","exec_price","px"],
    "size": ["size","qty","quantity","order_size","size_tokens","size_usd"],
    "side": ["side","direction","buy_sell","action"],
    "closedpnl": ["closedpnl","closed_pnl","pnl","realized_pnl"],
    "leverage": ["leverage","lev","leverage_x"],
    "order_id": ["order_id", "orderid"]
}
found = {}
for target, candidates in mappings.items():
    col = find_col_by_candidates(hist.columns, candidates)
    if col:
        found[col] = target

hist = hist.rename(columns=found)
print("Detected / renamed columns:", found)

for col in ["execution_price", "size", "closedpnl", "leverage"]:
    if col in hist.columns:
        hist[col] = pd.to_numeric(hist[col], errors="coerce")

if hist_date_col != "date":
    hist = hist.rename(columns={hist_date_col: "date"})

fg["date_only"] = fg["date"].dt.normalize()
hist["date_only"] = hist["date"].dt.normalize()
fg = fg.dropna(subset=["date_only"])
hist = hist.dropna(subset=["date_only"])

hist = hist.sort_values("date_only")
fg = fg.sort_values("date_only")

merged = pd.merge_asof(
    hist,
    fg[["date_only", "classification"]],
    on="date_only",
    direction="nearest"
)
print("Merged dataset shape:", merged.shape, "Missing sentiments:", merged["classification"].isna().sum())

if "size" in merged.columns and "closedpnl" in merged.columns:
    merged["profit_margin"] = np.where(merged["size"].notna() & merged["closedpnl"].notna(), merged["closedpnl"] / merged["size"], np.nan)
if "size" in merged.columns and "leverage" in merged.columns:
    merged["risk"] = np.where(merged["size"].notna() & merged["leverage"].notna(), merged["size"] * merged["leverage"], np.nan)
merged["sentiment_numeric"] = merged["classification"].map({"Fear":0,"Greed":1})

agg_dict = {"trades_count": ("account","count")}
if "closedpnl" in merged.columns:
    agg_dict.update({"avg_closedpnl": ("closedpnl","mean"), "median_closedpnl": ("closedpnl","median")})
if "leverage" in merged.columns:
    agg_dict.update({"avg_leverage": ("leverage","mean")})
if "size" in merged.columns:
    agg_dict.update({"median_size": ("size","median")})
if "profit_margin" in merged.columns:
    agg_dict.update({"avg_profit_margin": ("profit_margin","mean")})

agg_by_sent = merged.groupby("classification").agg(**agg_dict).reset_index()
print("Aggregations:\n", agg_by_sent)

csv_path = os.path.join(csv_out_dir, "processed_data.csv")
merged.to_csv(csv_path, index=False)

plots = []
if "closedpnl" in merged.columns and merged["classification"].notna().any():
    fig, ax = plt.subplots(figsize=(6, 4))
    data_fear = merged[merged["classification"] == "Fear"]["closedpnl"].dropna()
    data_greed = merged[merged["classification"] == "Greed"]["closedpnl"].dropna()
    ax.boxplot([data_fear, data_greed], labels=["Fear", "Greed"])
    ax.set_title("Closed PnL by Sentiment")
    ax.set_ylabel("Closed PnL")
    p1 = os.path.join(out_dir, "boxplot_closedpnl_by_sentiment.png")
    fig.savefig(p1, bbox_inches="tight")
    plt.close(fig)
    plots.append(p1)

if "leverage" in merged.columns:
    for sent in ["Fear", "Greed"]:
        vals = merged.loc[merged["classification"] == sent, "leverage"].dropna()
        if len(vals) > 0:
            fig, ax = plt.subplots(figsize=(6, 4))
            ax.hist(vals, bins=30)
            ax.set_title(f"Leverage distribution - {sent}")
            ax.set_xlabel("Leverage")
            ax.set_ylabel("Count")
            p = os.path.join(out_dir, f"hist_leverage_{sent.lower()}.png")
            fig.savefig(p, bbox_inches="tight")
            plt.close(fig)
            plots.append(p)

if "size" in merged.columns and "closedpnl" in merged.columns and "sentiment_numeric" in merged.columns:
    fig, ax = plt.subplots(figsize=(8, 6))
    scatter = ax.scatter(merged["size"], merged["closedpnl"], c=merged["sentiment_numeric"], cmap='viridis', alpha=0.6)
    ax.set_title("Closed PnL vs Trade Size (by Sentiment)")
    ax.set_xlabel("Trade Size")
    ax.set_ylabel("Closed PnL")

    legend1 = ax.legend(handles=scatter.legend_elements()[0], labels=["Fear", "Greed"], title="Sentiment")
    ax.add_artist(legend1)

    p_new = os.path.join(out_dir, "scatter_pnl_vs_size_by_sentiment.png")
    fig.savefig(p_new, bbox_inches="tight")
    plt.close(fig)
    plots.append(p_new)


numcols = merged.select_dtypes(include=[np.number]).columns.tolist()
if len(numcols) >= 2:
    corr = merged[numcols].corr()
    fig, ax = plt.subplots(figsize=(6, 6))
    cax = ax.matshow(corr, cmap="coolwarm")
    fig.colorbar(cax)
    ax.set_xticks(range(len(numcols)))
    ax.set_yticks(range(len(numcols)))
    ax.set_xticklabels(numcols, rotation=90)
    ax.set_yticklabels(numcols)
    ax.set_title("Correlation Matrix")
    p = os.path.join(out_dir, "corr_matrix.png")
    fig.savefig(p, bbox_inches="tight")
    plt.close(fig)
    plots.append(p)

print("Saved plots:", plots)

pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=15)
pdf.add_page()
pdf.set_font("Arial", size=12)
pdf.cell(0, 8, "Data Science Assignment - Trader Behavior vs Market Sentiment", ln=True)
pdf.ln(4)
pdf.set_font("Arial", size=10)
pdf.multi_cell(0, 6, "This report contains automatic aggregates and saved visualizations from the processed datasets. See images appended.")

pdf.ln(4)
pdf.set_font("Arial", size=9)
if not agg_by_sent.empty:
    for _, row in agg_by_sent.iterrows():
        avg_closedpnl_val = row.get('avg_closedpnl', 'NA')
        avg_leverage_val = row.get('avg_leverage', 'NA')

        avg_closedpnl_str = f"{avg_closedpnl_val:.4f}" if isinstance(avg_closedpnl_val, (int, float)) else avg_closedpnl_val
        avg_leverage_str = f"{avg_leverage_val:.3f}" if isinstance(avg_leverage_val, (int, float)) else avg_leverage_val

        pdf.multi_cell(0, 6, f"{row['classification']}: trades={int(row['trades_count'])}, avg_closedpnl={avg_closedpnl_str}, avg_leverage={avg_leverage_str}")

for p in plots:
    try:
        pdf.add_page()
        pdf.image(p, x=10, y=25, w=190)
    except Exception as e:
        print("Warning: could not add image to PDF:", p, e)

pdf.output(pdf_path)
print("Saved PDF:", pdf_path)

print("Done. Files created:")
print("Processed CSV:", csv_path)
print("Outputs directory:", out_dir)
print("PDF report:", pdf_path)

Loaded FG: (2644, 4) HIST: (211224, 16)
FG sentiment counts:
 classification
Fear     781
Greed    633
Name: count, dtype: int64
Detected / renamed columns: {'account': 'account', 'coin': 'symbol', 'execution_price': 'execution_price', 'size_tokens': 'size', 'side': 'side', 'closed_pnl': 'closedpnl', 'order_id': 'order_id'}


/tmp/ipython-input-2171860967.py:51: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  fg[fg_date_col] = pd.to_datetime(fg[fg_date_col], errors="coerce", infer_datetime_format=True)
/tmp/ipython-input-2171860967.py:52: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  hist[hist_date_col] = pd.to_datetime(hist[hist_date_col], errors="coerce", infer_datetime_format=True)


Merged dataset shape: (211224, 18) Missing sentiments: 0
Aggregations:
   classification  trades_count  avg_closedpnl  median_closedpnl  median_size  \
0           Fear        211224      48.749001               0.0         32.0   

   avg_profit_margin  
0           18.30723  


/tmp/ipython-input-2171860967.py:148: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  ax.boxplot([data_fear, data_greed], labels=["Fear", "Greed"])
/tmp/ipython-input-2171860967.py:178: UserWarning: Mismatched number of handles and labels: len(handles) = 1 len(labels) = 2
  legend1 = ax.legend(handles=scatter.legend_elements()[0], labels=["Fear", "Greed"], title="Sentiment")


Saved plots: ['./outputs/boxplot_closedpnl_by_sentiment.png', './outputs/scatter_pnl_vs_size_by_sentiment.png', './outputs/corr_matrix.png']
Saved PDF: ./outputs/ds_report.pdf
Done. Files created:
Processed CSV: ./csv_files/processed_data.csv
Outputs directory: ./outputs
PDF report: ./outputs/ds_report.pdf
